# MPIA Arxiv on Deck 2

Contains the steps to produce the paper extractions.

In [1]:
# Imports
import os
from IPython.display import Markdown, display
from tqdm.notebook import tqdm
import warnings
from PIL import Image 

# requires arxiv_on_deck_2

from arxiv_on_deck_2.arxiv2 import (get_new_papers, 
                                    get_paper_from_identifier,
                                    retrieve_document_source, 
                                    get_markdown_badge)
from arxiv_on_deck_2 import (latex,
                             latex_bib,
                             mpia,
                             highlight_authors_in_list)

# Sometimes images are really big
Image.MAX_IMAGE_PIXELS = 1000000000 

In [2]:
# Some useful definitions.

class AffiliationWarning(UserWarning):
    pass

class AffiliationError(RuntimeError):
    pass

def validation(source: str):
    """Raises error paper during parsing of source file
    
    Allows checks before parsing TeX code.
    
    Raises AffiliationWarning
    """
    check = mpia.affiliation_verifications(source, verbose=True)
    if check is not True:
        raise AffiliationError("mpia.affiliation_verifications: " + check)

        
warnings.simplefilter('always', AffiliationWarning)


def get_markdown_qrcode(paper_id: str):
    """ Generate a qrcode to the arxiv page using qrserver.com
    
    :param paper: Arxiv paper
    :returns: markdown text
    """
    url = r"https://api.qrserver.com/v1/create-qr-code/?size=100x100&data="
    txt = f"""<img src={url}"https://arxiv.org/abs/{paper_id}">"""
    txt = '<div id="qrcode">' + txt + '</div>'
    return txt

## get list of arxiv paper candidates

We use the MPIA mitarbeiter list webpage from mpia.de to get author names
We then get all new papers from Arxiv and match authors

In [3]:
# get list from MPIA website
# it automatically filters identified non-scientists :func:`mpia.filter_non_scientists`
mpia_authors = mpia.get_mpia_mitarbeiter_list()
normed_mpia_authors = [k[1] for k in mpia_authors]   # initials + fullname
new_papers = get_new_papers()
# add manual references
add_paper_refs = []
new_papers.extend([get_paper_from_identifier(k) for k in add_paper_refs])

candidates = []
for paperk in new_papers:
    # Check author list with their initials
    normed_author_list = [mpia.get_initials(k) for k in paperk['authors']]
    hl_authors = highlight_authors_in_list(normed_author_list, normed_mpia_authors, verbose=True)
    matches = [(hl, orig) for hl, orig in zip(hl_authors, paperk['authors']) if 'mark' in hl]
    paperk['authors'] = hl_authors
    if matches:
        # only select paper if an author matched our list
        candidates.append(paperk)
print("""Arxiv has {0:,d} new papers today""".format(len(new_papers)))        
print("""          {0:,d} with possible author matches""".format(len(candidates)))

U. Dudzeviciute  ->  U. Dudzeviciute  |  ['U. Dudzeviciute']
A. Gould  ->  A. Gould  |  ['A. Gould']
X. Zhang  ->  X. Zhang  |  ['X. Zhang']
X. Zhang  ->  X. Zhang  |  ['X. Zhang']
X. Zhang  ->  X. Zhang  |  ['X. Zhang']
X. Zhang  ->  X. Zhang  |  ['X. Zhang']
X. Zhang  ->  X. Zhang  |  ['X. Zhang']
X. Zhang  ->  X. Zhang  |  ['X. Zhang']
X. Zhang  ->  X. Zhang  |  ['X. Zhang']
X. Zhang  ->  X. Zhang  |  ['X. Zhang']


Arxiv has 64 new papers today
          10 with possible author matches


# Parse sources and generate relevant outputs

From the candidates, we do the following steps:
* get their tarball from ArXiv (and extract data)
* find the main .tex file: find one with \documentclass{...} (sometimes it's non trivial)
* Check affiliations with :func:`validation`, which uses :func:`mpia.affiliation_verifications`
* If passing the affiliations: we parse the .tex source
   * inject sub-documents into the main (flatten the main document)
   * parse structure, extract information (title, abstract, authors, figures...)
   * handles `\graphicspath` if provided
* Generate the .md document.

In [4]:
documents = []
failed = []
for paper in tqdm(candidates):
    paper_id = paper['identifier'].lower().replace('arxiv:', '')
    
    folder = f'tmp_{paper_id}'

    try:
        if not os.path.isdir(folder):
            folder = retrieve_document_source(f"{paper_id}", f'tmp_{paper_id}')
        
        try:
            doc = latex.LatexDocument(folder, validation=validation)    
        except AffiliationError as affilerror:
            msg = f"ArXiv:{paper_id:s} is not an MPIA paper... " + str(affilerror)
            failed.append((paper, "affiliation error: " + str(affilerror) ))
            continue
        
        # Hack because sometimes author parsing does not work well
        if (len(doc.authors) != len(paper['authors'])):
            doc._authors = paper['authors']
        else:
            # highlight authors (FIXME: doc.highlight_authors)
            # done on arxiv paper already
            doc._authors = highlight_authors_in_list(
                [mpia.get_initials(k) for k in doc.authors], 
                normed_mpia_authors, verbose=True)
        if (doc.abstract) in (None, ''):
            doc._abstract = paper['abstract']
            
        doc.comment = (get_markdown_badge(paper_id) + 
                       "<mark>Appeared on: " + paper['date'] + "</mark> - ")
        if paper['comments']:
            doc.comment += " _" + paper['comments'] + "_"
        
        full_md = doc.generate_markdown_text()
        
        full_md += get_markdown_qrcode(paper_id)
        
        # replace citations
        try:
            bibdata = latex_bib.LatexBib.from_doc(doc)
            full_md = latex_bib.replace_citations(full_md, bibdata)
        except Exception as e:
            print(e)
        
        documents.append((paper_id, full_md))
    except Exception as e:
        warnings.warn(latex.LatexWarning(f"{paper_id:s} did not run properly\n" +
                                         str(e)
                                        ))
        failed.append((paper, "latex error " + str(e)))

  0%|          | 0/10 [00:00<?, ?it/s]

Retrieving document from  https://arxiv.org/e-print/2402.07989


extracting tarball to tmp_2402.07989...

 done.


list index out of range
Retrieving document from  https://arxiv.org/e-print/2402.08116


...

 done.
Retrieving document from  https://arxiv.org/e-print/2402.08459


extracting tarball to tmp_2402.08459...

 done.
Retrieving document from  https://arxiv.org/e-print/2402.08460


extracting tarball to tmp_2402.08460...

 done.


### Export the logs

Throughout, we also keep track of the logs per paper. see `logs-{today date}.md` 

In [5]:
import datetime
today = str(datetime.date.today())
logfile = f"_build/html/logs/log-{today}.md"


with open(logfile, 'w') as logs:
    # Success
    logs.write(f'# Arxiv on Deck 2: Logs - {today}\n\n')
    logs.write("""* Arxiv had {0:,d} new papers\n""".format(len(new_papers)))
    logs.write("""    * {0:,d} with possible author matches\n\n""".format(len(candidates)))
    logs.write("## Sucessful papers\n\n")
    display(Markdown("## Successful papers"))
    success = [k[0] for k in documents]
    for candid in candidates:
        if candid['identifier'].split(':')[-1] in success:
            display(candid)
            logs.write(candid.generate_markdown_text() + '\n\n')

    ## failed
    logs.write("## Failed papers\n\n")
    display(Markdown("## Failed papers"))
    failed = sorted(failed, key=lambda x: x[1])
    current_reason = ""
    for paper, reason in failed:
        if 'affiliation' in reason:
            color = 'green'
        else:
            color = 'red'
        data = Markdown(
                paper.generate_markdown_text() + 
                f'\n|<p style="color:{color:s}"> **ERROR** </p>| <p style="color:{color:s}">{reason:s}</p> |'
               )
        if reason != current_reason:
            logs.write(f'### {reason:s} \n\n')
            current_reason = reason
        logs.write(data.data + '\n\n')
        
        # only display here the important errors (all in logs)
        # if color in ('red',):
        display(data)

## Successful papers


|||
|---:|:---|
| [![arXiv](https://img.shields.io/badge/arXiv-arXiv:2402.07989-b31b1b.svg)](https://arxiv.org/abs/arXiv:2402.07989) | **A new census of dust and PAHs at z=0.7-2 with JWST MIRI**  |
|| I. Shivaei, et al. -- incl., <mark>U. Dudzeviciute</mark> |
|*Appeared on*| *2024-02-14*|
|*Comments*| *Submitted to A&A*|
|**Abstract**| This paper utilizes the JWST MIRI multi-band imaging data from the SMILES survey (5-25micron), complemented with HST and NIRCam photometric and spectroscopic data from the JADES and FRESCO surveys for 443 star-forming (non-AGN) galaxies at z=0.7-2.0 to extend the study of dust and PAH emission to a new mass and SFR parameter space beyond our local universe. We find a strong correlation between the fraction of dust in PAHs (PAH fraction, q_PAH) with stellar mass. Moreover, the PAH fraction behavior as a function of gas-phase metallicity is similar to that at z~0 from previous studies, suggesting a universal relation: q_PAH is constant (~3.4%) above a metallicity of ~ 0.5$Z_{\odot}$ and decreases to <1% at metallicities $<0.3Z_{\odot}$. This indicates that metallicity is a good indicator of the ISM properties that affect the balance between the formation and destruction of PAHs. The lack of a redshift evolution from z~0-2 also implies that above $0.5\,Z_{\odot}$, the PAH emission effectively traces obscured luminosity and the previous locally-calibrated PAH-SFR calibrations remain applicable in this metallicity regime. We observe a strong correlation between obscured UV luminosity fraction (ratio of obscured to total luminosity) and stellar mass. Above the stellar mass of $>5\times 10^9M_{\odot}$, on average, more than half of the emitted luminosity is obscured, while there exists a non-negligible population of lower mass galaxies with >50% obscured fractions. At a fixed mass, the obscured fraction correlates with SFR surface density. This is a result of higher dust covering fractions in galaxies with more compact star forming regions. Similarly, galaxies with high IRX (IR to UV luminosity) at a given mass or UV continuum slope tend to have higher SFR surface density and shallower attenuation curves, owing to their higher effective dust optical depths and more compact star forming regions. |

## Failed papers


|||
|---:|:---|
| [![arXiv](https://img.shields.io/badge/arXiv-arXiv:2402.08456-b31b1b.svg)](https://arxiv.org/abs/arXiv:2402.08456) | **The SRG/eROSITA All-Sky Survey: Weak-Lensing of eRASS1 Galaxy Clusters  in KiDS-1000 and Consistency Checks with DES Y3 & HSC-Y3**  |
|| F. Kleinebreil, et al. -- incl., <mark>X. Zhang</mark> |
|*Appeared on*| *2024-02-14*|
|*Comments*| *22 pages, 23 figures*|
|**Abstract**| We aim to participate in the calibration of the X-ray photon count rate to halo mass scaling relation of galaxy clusters selected in the first eROSITA All-Sky Survey on the Western Galactic Hemisphere (eRASS1) using KiDS-1000 weak-lensing (WL) data. We measure the radial shear profiles around eRASS1 galaxy clusters using background galaxies in KiDS-1000, as well as the cluster member contamination. Furthermore we provide consistency checks with the other stage-III WL surveys who take part in the eRASS1 mass calibration, DES Y3 and HSC-Y3. We determine the cluster member contamination of eRASS1 clusters present in KiDS-1000 based on source number density profiles, where we account for the obscuration caused by cluster galaxies. The extracted shear profiles, together with the contamination model and the lens sample selection, are then analysed through a Bayesian population model. We calibrate the WL mass bias parameter by analysing realistic synthetic shear profiles from mock cluster catalogues. Our consistency checks between KiDS-1000 and DES Y3 & HSC-Y3 include the comparison of contamination-corrected density contrast profiles employing the union of background sources around common clusters, as well as the individual scaling relation results. We present a global contamination model for eRASS1 clusters in KiDS-1000 and the calibration results of the X-ray photon count rate to halo mass relation. The results of the WL mass bias parameter show that the uncertainty of the multiplicative shear bias dominates the systematic error budget at low clusters redshifts while the uncertainty of our contamination model does at high ones. The cross-checks between the three WL surveys show that they are statistically consistent with each other. This enables for the first time cosmological constraints from clusters calibrated by three state-of-the-art WL surveys. (abridged) |
|<p style="color:green"> **ERROR** </p>| <p style="color:green">affiliation error: mpia.affiliation_verifications: '69117' keyword not found.</p> |


|||
|---:|:---|
| [![arXiv](https://img.shields.io/badge/arXiv-arXiv:2402.08452-b31b1b.svg)](https://arxiv.org/abs/arXiv:2402.08452) | **The SRG/eROSITA All-Sky Survey: The first catalog of galaxy clusters and  groups in the Western Galactic Hemisphere**  |
|| E. Bulbul, et al. -- incl., <mark>X. Zhang</mark> |
|*Appeared on*| *2024-02-14*|
|*Comments*| *in press in A&A. 27 pages, 21 figures, 3 tables*|
|**Abstract**| Clusters of galaxies can be used as powerful probes to study astrophysical processes on large scales, test theories of the growth of structure, and constrain cosmological models. The driving science goal of the SRG/eROSITA All-Sky Survey (eRASS) is to assemble a large sample of X-ray-selected clusters with a well-defined selection function to determine the evolution of the mass function and, hence, the cosmological parameters. We present here a catalog of 12247 optically confirmed galaxy groups and clusters detected in the 0.2-2.3 keV as extended X-ray sources in a 13,116deg$^2$ region in the western Galactic hemisphere of the sky, which eROSITA surveyed in its first six months of operation. The clusters in the sample span the redshift range $0.003<z<1.32$. The majority (68%) of these clusters, 8361 sources, represent new discoveries without known counterparts in the literature. The mass range of the sample covers three orders of magnitude from $5\times10^{12}M_{\rm sun}$ to $2\times10^{15}M_{\rm sun}$. We construct a sample for cosmology with a higher purity level (~95%) than the primary sample, comprising 5259 securely detected and confirmed clusters in the 12791deg$^{2}$ common footprint with the DESI Legacy Survey DR10. We characterize the X-ray properties of each cluster, including their flux, luminosity and temperature, the total mass, gas mass, gas mass fraction, and mass proxy $Y_{X}$. These are determined within two apertures, 300 kpc, and the overdensity radius $R_{500}$, and are calculated by applying a forward modeling approach with a rigorous X-ray background treatment, K-factor, and the Galactic absorption corrections. Population studies utilizing LogN-LogS, the number of clusters detected above a given flux limit, and the luminosity function show overall agreement with the previous X-ray surveys after accounting for the survey completeness and purity (ABRIDGED) |
|<p style="color:green"> **ERROR** </p>| <p style="color:green">affiliation error: mpia.affiliation_verifications: 'Heidelberg' keyword not found.</p> |

...


|||
|---:|:---|
| [![arXiv](https://img.shields.io/badge/arXiv-arXiv:2402.08457-b31b1b.svg)](https://arxiv.org/abs/arXiv:2402.08457) | **The SRG/eROSITA All-Sky Survey: X-ray selection function models for the  eRASS1 galaxy cluster cosmology**  |
|| N. Clerc, et al. -- incl., <mark>X. Zhang</mark> |
|*Appeared on*| *2024-02-14*|
|*Comments*| *24 pages, 31 figures, 2 tables. Submitted to A&A*|
|**Abstract**| Characterising galaxy cluster populations from catalog of sources selected in astronomical surveys requires knowledge of sample incompleteness, known as selection function. The first All-Sky Survey (eRASS1) by eROSITA onboard Spectrum Roentgen Gamma (SRG) has enabled the collection of large samples of galaxy clusters detected in the soft X-ray band over the Western Galactic hemisphere. The driving goal consists in constraining cosmological parameters, which puts stringent requirements on accuracy, flexibility and explainability of the selection function models. We use a large set of mock observations of the eRASS1 survey and we process simulated data identically to the real eRASS1 events. We match detected sources to simulated clusters and we associate detections to intrinsic cluster properties. We train a series of models to build selection functions depending only on observable surface brightness data. We develop a second series of models relying on global cluster characteristics such as X-ray luminosity, flux, and expected instrumental count-rate as well as on morphological properties. We validate our models using our simulations and we rank them according to selected performance metrics. We validate the models with datasets of clusters detected in X-rays and via the Sunyaev-Zeldovich effect. We present the complete Bayesian population modelling framework developed for this purpose. Our results reveal the surface brightness characteristics most relevant to cluster selection in the eRASS1 sample, in particular the ambiguous role of central surface brightness at the scale of the instrument resolution. We have produced a series of user-friendly selection function models and demonstrated their validity and their limitations. Our selection function for bright sources reproduces well the catalog matches with external datasets. (abridged) |
|<p style="color:green"> **ERROR** </p>| <p style="color:green">affiliation error: mpia.affiliation_verifications: 'Heidelberg' keyword not found.</p> |


|||
|---:|:---|
| [![arXiv](https://img.shields.io/badge/arXiv-arXiv:2402.08458-b31b1b.svg)](https://arxiv.org/abs/arXiv:2402.08458) | **The SRG/eROSITA All-Sky Survey: Cosmology Constraints from Cluster  Abundances in the Western Galactic Hemisphere**  |
|| V. Ghirardini, et al. -- incl., <mark>X. Zhang</mark> |
|*Appeared on*| *2024-02-14*|
|*Comments*| *43 pages, 22 figures, submitted to A&A*|
|**Abstract**| The cluster mass function traces the growth of linear density perturbations and provides valuable insights into the growth of structures, the nature of dark matter, and the cosmological parameters governing the Universe. The primary science goal of eROSITA, on board the {\it Spectrum Roentgen Gamma (SRG)} mission, launched in 2019, is to constrain cosmology through the evolution of cluster mass function. In this paper, we present the cosmological constraints obtained from 5259 clusters of galaxies detected over an area of 12791~deg$^2$ in the Western Galactic Hemisphere of the eROSITA's first All-Sky Survey (eRASS1). The common footprint region between the eROSITA Survey and DES, KiDS, and HSC surveys is used for calibration of the scaling between X-ray count rate and their total mass through measurements of their weak gravitational lensing signal. eRASS1 cluster abundances constrain the $\Lambda$CDM parameters, which are the energy density of the total matter to $\Omega_{\mathrm{m}}=0.29^{+0.01}_{-0.02}$, and the normalization of the density fluctuations to $\sigma_8=0.88\pm0.02$ and their combination yields $S_8=\sigma_8 (\Omega_\mathrm{m} / 0.3)^{0.5}=0.86\pm0.01$, consistent and at a similar precision with the state-of-the-art CMB measurements. eRASS1 cosmological experiment places a most stringent upper limit on the summed masses of left-handed light neutrinos to $\sum m_\nu< 0.22\mathrm{~eV}$ (95\% confidence interval). Combining eRASS1 cluster abundance measurements with CMB and ground-based neutrino oscillation experiments, we measure the summed neutrino masses to be $\sum m_\nu=0.08_{-0.02}^{+0.03}\mathrm{~eV}$ or $\sum m_\nu=0.12_{-0.01}^{+0.03}\mathrm{~eV}$ depending on the mass hierarchy scenario for neutrino eigenstates. eRASS1 cluster abundances significantly improve the constraints on the dark energy equation of state parameter to $w=-1.12\pm0.12$. (ABRIDGED) |
|<p style="color:green"> **ERROR** </p>| <p style="color:green">affiliation error: mpia.affiliation_verifications: 'Heidelberg' keyword not found.</p> |


|||
|---:|:---|
| [![arXiv](https://img.shields.io/badge/arXiv-arXiv:2402.08459-b31b1b.svg)](https://arxiv.org/abs/arXiv:2402.08459) | **The SRG-eROSITA All-Sky Survey : Constraints on f(R) Gravity from  Cluster Abundance**  |
|| E. Artis, et al. -- incl., <mark>X. Zhang</mark> |
|*Appeared on*| *2024-02-14*|
|*Comments*| *13 pages, 6 figures, submitted to A&A*|
|**Abstract**| The evolution of the cluster mass function traces the growth of the linear density perturbations and can be utilized for constraining the parameters of cosmological and alternative gravity models. In this context, we present new constraints on potential deviations from general relativity by investigating the Hu-Sawicki parametrization of the f(R) gravity with the first SRG-eROSITA All-Sky Survey (eRASS1) cluster catalog in the Western Galactic Hemisphere in combination with the overlapping Dark Energy Survey Year 3, KiloDegree Survey and Hyper Supreme Camera data for weak lensing mass calibration. For the first time, we present constraints obtained from cluster abundances only. When we consider massless neutrinos, we find a strict upper limit of log |fR0| < -4.31 at 95% confidence level. Massive neutrinos suppress structure growth at small scales, and thus have the opposite effect of f(R) gravity. We consequently investigate the joint fit of the mass of the neutrinos with the modified gravity parameter. We obtain log |fR0| < -4.12 jointly with \sum m_\nu < 0.44 e.V. at 95% confidence level, tighter than the limits in the literature utilizing cluster counts only. At log |fR0|= - 6, the number of clusters is not significantly changed by the theory. Consequently, we do not find any statistical deviation from general relativity from the study of eRASS1 cluster abundance. Deeper surveys with eROSITA, increasing the number of detected clusters, will further improve constraints on log |fR0| and investigate alternative gravity theories. |
|<p style="color:green"> **ERROR** </p>| <p style="color:green">affiliation error: mpia.affiliation_verifications: 'Heidelberg' keyword not found.</p> |


|||
|---:|:---|
| [![arXiv](https://img.shields.io/badge/arXiv-arXiv:2402.08460-b31b1b.svg)](https://arxiv.org/abs/arXiv:2402.08460) | **The SRG/eROSITA All-Sky Survey: Tracing the Large-Scale Structure with a  clustering study of galaxy clusters**  |
|| R. Seppi, et al. -- incl., <mark>X. Zhang</mark> |
|*Appeared on*| *2024-02-14*|
|*Comments*| *Submitted to A&A, 17 pages, 15 figures*|
|**Abstract**| The spatial distribution of galaxy clusters provides a reliable tracer of the large-scale distribution of matter in the Universe. The clustering signal depends on intrinsic cluster properties and cosmological parameters. The ability of eROSITA onboard Spectrum-Roentgen-Gamma (SRG) to discover galaxy clusters allows probing the association of extended X-ray emission to dark matter haloes. We aim to measure the projected two-point correlation function to study the occupation of dark matter halos by clusters and groups detected by the first eROSITA all-sky survey (eRASS1). We create five volume-limited samples probing clusters with different redshift and X-ray luminosity. We interpret the correlation function with halo occupation distribution (HOD) and halo abundance matching (HAM) models. We simultaneously fit cosmological parameters and halo bias of a flux-limited sample of 6493 clusters with purity > 96%. Results. We obtain a detailed view of the halo occupation for eRASS1 clusters. The fainter population at low redshift (S0: LX = 4.63E43 erg/s, 0.1 < z < 0.2) is the least biased compared to dark matter, with b = 2.95 $\pm$ 0.21. The brightest clusters up to higher redshift (S4: LX = 1.77E44 erg/s , 0.1 < z < 0.6) exhibit a higher bias b = 4.34 $\pm$ 0.62. Satellite groups are rare, with a satellite fraction < 14.9% (8.1) for the S0 (S4) sample. We combine the HOD prediction with a HAM procedure to constrain the scaling relation between LX and mass in a new way and find a scatter of 0.36. We obtain cosmological constraints for the physical cold dark matter density 0.12+0.03-0.02 and an average halo bias b = 3.63+1.02-0.85. We model the clustering of galaxy clusters with a HOD approach for the first time, paving the way for future studies combining eROSITA with 4MOST, SDSS, Euclid, Rubin, and DESI to unravel the cluster distribution in the Universe. |
|<p style="color:green"> **ERROR** </p>| <p style="color:green">affiliation error: mpia.affiliation_verifications: 'Heidelberg' keyword not found.</p> |


|||
|---:|:---|
| [![arXiv](https://img.shields.io/badge/arXiv-arXiv:2402.08116-b31b1b.svg)](https://arxiv.org/abs/arXiv:2402.08116) | **OGLE-2023-BLG-0836L: The sixth microlensing planet in a binary stellar  system**  |
|| C. Han, et al. -- incl., <mark>A. Gould</mark> |
|*Appeared on*| *2024-02-14*|
|*Comments*| *11 pages, 12 figures*|
|**Abstract**| Light curves of microlensing events occasionally deviate from the smooth and symmetric form of a single-lens single-source event. While most of these anomalous events can be accounted for by employing a binary-lens single-source (2L1S) or a single-lens binary-source (1L2S) framework, it is established that a small fraction of events remain unexplained by either of these interpretations. We carry out a project in which data collected by high-cadence microlensing surveys were reinvestigated with the aim of uncovering the nature of anomalous lensing events with no proposed 2L1S or 1L2S models. From the project, we find that the anomaly appearing in the lensing event OGLE-2023-BLG-0836 cannot be explained by the usual interpretations and conduct a comprehensive analysis of the event. From thorough modeling of the light curve under sophisticated lens-system configurations, we have arrived at the conclusion that a triple-mass lens system is imperative to account for the anomaly features observed in the lensing light curve. From the Bayesian analysis using the measured observables of the event time scale and angular Einstein radius, we determine that the least massive component of the lens has a planetary mass of $4.36^{+2.35}_{-2.18}~M_{\rm J}$. This planet orbits within a stellar binary system composed of two stars with masses $0.71^{+0.38}_{-0.36}~M_\odot$ and $0.56^{+0.30}_{-0.28}~M_\odot$. This lensing event signifies the sixth occurrence of a planetary microlensing system in which a planet belongs to a stellar binary system. |
|<p style="color:red"> **ERROR** </p>| <p style="color:red">latex error Unable to locate Ghostscript on paths</p> |

## Export documents

We now write the .md files and export relevant images

In [6]:
def export_markdown_summary(md: str, md_fname:str, directory: str):
    """Export MD document and associated relevant images"""
    import os
    import shutil
    import re

    if (os.path.exists(directory) and not os.path.isdir(directory)):
        raise RuntimeError(f"a non-directory file exists with name {directory:s}")

    if (not os.path.exists(directory)):
        print(f"creating directory {directory:s}")
        os.mkdir(directory)

    fig_fnames = (re.compile(r'\[Fig.*\]\((.*)\)').findall(md) + 
                  re.compile(r'\<img src="([^>\s]*)"[^>]*/>').findall(md))
    for fname in fig_fnames:
        if 'http' in fname:
            # No need to copy online figures
            continue
        destdir = os.path.join(directory, os.path.dirname(fname))
        destfname = os.path.join(destdir, os.path.basename(fname))
        try:
            os.makedirs(destdir)
        except FileExistsError:
            pass
        shutil.copy(fname, destfname)
    with open(os.path.join(directory, md_fname), 'w') as fout:
        fout.write(md)
    print("exported in ", os.path.join(directory, md_fname))
    [print("    + " + os.path.join(directory,fk)) for fk in fig_fnames]

In [7]:
for paper_id, md in documents:
    export_markdown_summary(md, f"{paper_id:s}.md", '_build/html/')

exported in  _build/html/2402.07989.md
    + _build/html/tmp_2402.07989/./figs/three-size.png
    + _build/html/tmp_2402.07989/./figs/obsc_frac_w17_errfunc.png
    + _build/html/tmp_2402.07989/./figs/obsc_frac_w17_errfunc-redshift.png
    + _build/html/tmp_2402.07989/./figs/qpah-mass.png
    + _build/html/tmp_2402.07989/./figs/f77-mass.png


## Display the papers

Not necessary but allows for a quick check.

In [8]:
[display(Markdown(k[1])) for k in documents];

<div class="macros" style="visibility:hidden;">
$\newcommand{\ensuremath}{}$
$\newcommand{\xspace}{}$
$\newcommand{\object}[1]{\texttt{#1}}$
$\newcommand{\farcs}{{.}''}$
$\newcommand{\farcm}{{.}'}$
$\newcommand{\arcsec}{''}$
$\newcommand{\arcmin}{'}$
$\newcommand{\ion}[2]{#1#2}$
$\newcommand{\textsc}[1]{\textrm{#1}}$
$\newcommand{\hl}[1]{\textrm{#1}}$
$\newcommand{\footnote}[1]{}$
$\newcommand{\Msun}{ M_{\odot}}$
$\newcommand{\Lsun}{ L_{\odot}}$
$\newcommand{\um}{ \mum}$
$\newcommand{\uJy}{ \muJy}$
$\newcommand{\qpah}{q_{\rm PAH}}$
$\newcommand{\logoh}{12+\log{\rm{(O/H)}}}$
$\newcommand{\fobsc}{f_{\rm obsc}}$
$\newcommand{\Ssfr}{\Sigma_{\rm SFR}}$
$\newcommand{\}{natexlab}$</div>



<div id="title">

# A new census of dust and PAHs at $z=0.7-2$ with JWST MIRI   

</div>
<div id="comments">

[![arXiv](https://img.shields.io/badge/arXiv-2402.07989-b31b1b.svg)](https://arxiv.org/abs/2402.07989)<mark>Appeared on: 2024-02-14</mark> -  _Submitted to A&A_

</div>
<div id="authors">

I. Shivaei, et al.

</div>
<div id="abstract">

**Abstract:** This paper utilizes the James Webb Space Telescope (JWST) Mid-Infrared Instrument (MIRI) to extend the observational studies of dust and Polycyclic Aromatic Hydrocarbon (PAH) emission to a new mass and star formation rate (SFR) parameter space beyond our local universe. The combination of fully sampled SEDs with multiple mid-IR bands and the unprecedented sensitivity of MIRI allows us to investigate dust obscuration and PAH behavior from $z=0.7$ up to $z=2$ in typical main-sequence galaxies. Our focus is on constraining the evolution of PAH strength and dust-obscured luminosity fraction before and during cosmic noon, the epoch of peak star formation activity in the universe. In this study, we utilize MIRI multi-band imaging data from the SMILES survey (5 to 25 ${\um}$ ), complemented with NIRCam photometry from the JADES survey (1 to 5 ${\um}$ ), available HST photometry (0.4 to 0.9 ${\um}$ ), and spectroscopic redshifts from the FRESCO and JADES surveys in GOODS-S for 443 star-forming (non-AGN) galaxies at $z=0.7-2.0$ . This redshift range is chosen to ensure that the MIRI data cover mid-IR dust emission. Our methodology involves employing UV-to-IR energy balance SED fitting to robustly constrain the fraction of dust mass in PAHs and dust obscured luminosity. Additionally, we infer dust sizes from MIRI 15 ${\um}$ imaging data, enhancing our understanding of the physical characteristics of dust within these galaxies. We find a strong correlation between the fraction of dust in PAHs (PAH fraction, ${\qpah}$ ) with stellar mass. Moreover, the PAH fraction behaviour as a function of gas-phase metallicity is similar to that at $z\sim 0$ from previous studies, suggesting a universal relation: ${\qpah}$ is constant ( $\sim 3.4$ \% ) above a metallicity of $Z \sim 0.5 Z_{\odot}$ and decreases to $<1$ \% at metallicities $\lesssim 0.3 Z_{\odot}$ . This indicates that metallicity is a good indicator of the ISM properties that affect the balance between the formation and destruction of PAHs. The lack of a redshift evolution from $z\sim 0-2$ also implies that above $Z\sim 0.5 Z_{\odot}$ , the PAH emission effectively traces obscured luminosity and the previous locally-calibrated PAH-SFR calibrations remain applicable in this metallicity regime. We observe a strong correlation between obscured UV luminosity fraction (ratio of obscured to total luminosity) and stellar mass. Above the stellar mass of $M_*>5\times 10^9$ ${\Msun}$ , on average, more than half of the emitted luminosity is obscured, while there exists a non-negligible population of lower mass galaxies with $>50\%$ obscured fractions. At a fixed mass, the obscured fraction correlates with SFR surface density. This is a result of higher dust covering fractions in galaxies with more compact star forming regions. Similarly, galaxies with high IRX (IR to UV luminosity) at a given mass or UV continuum slope ( $\beta$ ) tend to have higher ${\Ssfr}$ and shallower attenuation curves, owing to their higher effective dust optical depths and more compact star forming regions.

</div>

<div id="div_fig1">

<img src="tmp_2402.07989/./figs/three-size.png" alt="Fig13" width="100%"/>

**Figure 13. -** Obscured luminosity fraction as a function of {$\Ssfr$}(using F1500W sizes) and F1500W effective radius (middle) for the F1500W spatially resolved sample (the filter FWHM is 0.5 arcsec). At a given mass, {$\fobsc$} increases with {$\Ssfr$} and decreases with rest-frame mid-IR (dust) radii. Right panel shows SFR versus F1500W size for the same sample. The near constant SFR with ${\rm R_{e,F1500W}}$ in each mass bin indicates that the relationship between {$\fobsc$} and size is independent of SFR.
    Individual measurements are shown with small symbols and color-coded in three stellar mass bins with the same number of galaxies, shown in the legend. In each mass bin, average values in bins of the parameter on the x-axis are shown with larger symbols with the corresponding shape and color of the mass bin. Pearson correlation coefficients ($\rho$) are shown in the corners of the plots with corresponding colors of each mass bin.  Linear fits to the average values of {$\fobsc$} versus $\log$({$\Ssfr$}) are shown in the left panel. The fit parameters are in Table \ref{table:fobsc}.  (*fig:obsc_frac*)

</div>
<div id="div_fig2">

<img src="tmp_2402.07989/./figs/obsc_frac_w17_errfunc.png" alt="Fig12.1" width="50%"/><img src="tmp_2402.07989/./figs/obsc_frac_w17_errfunc-redshift.png" alt="Fig12.2" width="50%"/>

**Figure 12. -** Obscured luminosity fraction as a function of stellar mass for the full sample (left) and separated by redshift (right). Obscured fraction is defined as the obscured UV to total (intrinsic) UV luminosity at 1550 {Å}, inferred from the best-fit UV-to-IR SEDs.
    We show the moving median (dashed thick lines), moving weighted-average (dotted line) and best-fit curve (solid lines) to the samples at $M_*>4\times10^8${$\Msun$}, where the data is mass-complete.
    In the left panel, we also show the \citet{whitaker17} fit to $z= 0.5-2.5$ with an orange curve. The \citet{whitaker17} study was based on Spitzer/MIPS data with a wider area but biased to more obscured systems.
    Our weighted-average trend is also dominated by the heavily obscured systems, and therefore similar to the \citep{whitaker17} curve. However, the median trend that is more representative of the full population captured by the deeper MIRI data (relative to the MIPS data) is systematically lower.
    Furthermore, as shown in the right panel, there is no redshift evolution in this relation in our sample.
     (*fig:obsc_frac_w17*)

</div>
<div id="div_fig3">

<img src="tmp_2402.07989/./figs/qpah-mass.png" alt="Fig8.1" width="50%"/><img src="tmp_2402.07989/./figs/f77-mass.png" alt="Fig8.2" width="50%"/>

**Figure 8. -** PAH mass fraction (left) and luminosity fraction (right) as a function of stellar mass. Moving median (black curve) is shown for the robust {$\qpah$} sample (see Section \ref{sec:final-sample} for the definition of samples), and all galaxies are color-coded by sSFR. In the left panel, vertical lines show the median PAH mass fractions of $z\sim 0$ galaxies with {$\logoh$}$<8.1$ and $>8.1$ from \citet{draine07b}. On the top x-axes, we also show the estimated metallicities from the O3N2-based mass-metallicity relation (MZR) of \cite{topping21} at $z\sim 1.5$, and assuming the \cite{pp04} calibration. The $z\sim 0.7-2$ sample has a very similar {$\qpah$}-metallicity behaviour to that inferred at $z\sim 0$.
     (*fig:pah-mass*)

</div><div id="qrcode"><img src=https://api.qrserver.com/v1/create-qr-code/?size=100x100&data="https://arxiv.org/abs/2402.07989"></div>

# Create HTML index

In [9]:
from datetime import datetime, timedelta, timezone
from glob import glob
import os

files = glob('_build/html/*.md')
days = 7
now = datetime.today()
res = []
for fk in files:
    stat_result = os.stat(fk).st_ctime
    modified = datetime.fromtimestamp(stat_result, tz=timezone.utc).replace(tzinfo=None)
    delta = now.today() - modified
    if delta <= timedelta(days=days):
        res.append((delta.seconds, fk))
res = [k[1] for k in reversed(sorted(res, key=lambda x:x[1]))]
npub = len(res)
print(len(res), f" publications files modified in the last {days:d} days.")
# [ print('\t', k) for k in res ];

371  publications files modified in the last 7 days.


In [10]:
import datetime
from glob import glob

def get_last_n_days(lst, days=1):
    """ Get the documents from the last n days """
    sorted_lst = sorted(lst, key=lambda x: x[1], reverse=True)
    for fname, date in sorted_lst:
        if date >= str(datetime.date.today() - datetime.timedelta(days=days)):
            yield fname

def extract_appearance_dates(lst_file):
    dates = []

    def get_date(line):
        return line\
            .split('Appeared on:')[-1]\
            .split('</mark>')[0].strip()

    for fname in lst:
        with open(fname, 'r') as f:
            found_date = False
            for line in f:
                if not found_date:
                    if "Appeared on" in line:
                        found_date = True
                        dates.append((fname, get_date(line)))
                else:
                    break
    return dates

from glob import glob
lst = glob('_build/html/*md')
days = 7
dates = extract_appearance_dates(lst)
res = list(get_last_n_days(dates, days))
npub = len(res)
print(len(res), f" publications in the last {days:d} days.")

4  publications in the last 7 days.


In [11]:
def create_carousel(npub=4):
    """ Generate the HTML code for a carousel with `npub` slides """
    carousel = ["""  <div class="carousel" """,
                """       data-flickity='{ "autoPlay": 10000, "adaptiveHeight": true, "resize": true, "wrapAround": true, "pauseAutoPlayOnHover": true, "groupCells": 1 }' id="asyncTypeset">"""
                ]
    
    item_str = """    <div class="carousel-cell"> <div id="slide{k}" class="md_view">Content {k}</div> </div>"""
    for k in range(1, npub + 1):
        carousel.append(item_str.format(k=k))
    carousel.append("  </div>")
    return '\n'.join(carousel)

def create_grid(npub=4):
    """ Generate the HTML code for a flat grid with `npub` slides """
    grid = ["""  <div class="grid"> """,
                ]
    
    item_str = """    <div class="grid-item"> <div id="slide{k}" class="md_view">Content {k}</div> </div>"""
    for k in range(1, npub + 1):
        grid.append(item_str.format(k=k))
    grid.append("  </div>")
    return '\n'.join(grid)

In [12]:
carousel = create_carousel(npub)
docs = ', '.join(['"{0:s}"'.format(k.split('/')[-1]) for k in res])
slides = ', '.join([f'"slide{k}"' for k in range(1, npub + 1)])

with open("daily_template.html", "r") as tpl:
    page = tpl.read()
    page = page.replace("{%-- carousel:s --%}", carousel)\
               .replace("{%-- suptitle:s --%}",  "7-day archives" )\
               .replace("{%-- docs:s --%}", docs)\
               .replace("{%-- slides:s --%}", slides)
    
with open("_build/html/index_7days.html", 'w') as fout:
    fout.write(page)

In [13]:
# redo for today
days = 1
res = list(get_last_n_days(dates, days))
npub = len(res)
print(len(res), f" publications in the last day.")

carousel = create_carousel(npub)
docs = ', '.join(['"{0:s}"'.format(k.split('/')[-1]) for k in res])
slides = ', '.join([f'"slide{k}"' for k in range(1, npub + 1)])

with open("daily_template.html", "r") as tpl:
    page = tpl.read()
    page = page.replace("{%-- carousel:s --%}", carousel)\
               .replace("{%-- suptitle:s --%}",  "Daily" )\
               .replace("{%-- docs:s --%}", docs)\
               .replace("{%-- slides:s --%}", slides)
    
# print(carousel, docs, slides)
# print(page)
with open("_build/html/index_daily.html", 'w') as fout:
    fout.write(page)

2  publications in the last day.


In [14]:
# Create the flat grid of the last N papers (fixed number regardless of dates)
from itertools import islice 

npub = 6
res = [k[0] for k in (islice(reversed(sorted(dates, key=lambda x: x[1])), 6))]
print(len(res), f" {npub} publications selected.")

grid = create_grid(npub)
docs = ', '.join(['"{0:s}"'.format(k.split('/')[-1]) for k in res])
slides = ', '.join([f'"slide{k}"' for k in range(1, npub + 1)])

with open("grid_template.html", "r") as tpl:
    page = tpl.read()
    page = page.replace("{%-- grid-content:s --%}", grid)\
               .replace("{%-- suptitle:s --%}",  f"Last {npub:,d} papers" )\
               .replace("{%-- docs:s --%}", docs)\
               .replace("{%-- slides:s --%}", slides)
    
# print(grid, docs, slides)
# print(page)
with open("_build/html/index_npub_grid.html", 'w') as fout:
    fout.write(page)

6  6 publications selected.
